Untuk melihat bagaimana efektifnya **transfer learning**, kita akan belajar menggunakan **transfer learning** pada dataset cheesman yang kita pelajari pada submodul sebelumnya.

Kode yang akan kita gunakan pada submodul ini hampir sama dengan kode pada latihan sebelumnya, jadi silahkan menyalin atau menggunakan kode padat latihan sebelumnya. Pada cell Colaboratory pertama kita unduh dataset chessman yang akan kita pakai.

In [1]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-09-20 14:39:49--  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip [following]
--2021-09-20 14:39:50--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M   222MB/s    in 

Kemudian, kita buat direktori untuk dataset kita untuk digunakan pada ImageDataGenerator. Pada ImageDataGenerator gunakan augmentasi yang sama dengan latihan pada submodul sebelumnya.

In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split

Bagi dataset menjadi data training dan data validasi.

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
    
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


Nah, disini kita mulai mengimplementasikan transfer learning. Untuk model yang kita pilih sebagai model dasar transfer learning adalah *ResNet152V2*. Model *ResNet152V2* memiliki sebanyak 152 layer dan tersedia di library keras. Kita dapat memanfaatkan fitur-fitur yang telah dipelajari oleh model tersebut untuk dipakai model kita. 

Untuk mengimplementasikan transfer learning sangatlah mudah seperti kode di bawah. Kita hanya perlu menambahkan 2 buah baris kode berbeda. Layer pertama pada model kita adalah model yang kita pakai untuk transfer learning. Kita cukup memanggil kelas `ResNet152V2` dan mengisi parameter sebagai berikut:
*   Weight : ini adalah bobot atau parameter seperti yang telah dibahas pada kelas machine learning pemula. Untuk parameter weight kita mengisi nilai ‘imagenet’. Artinya kita ingin menggunakan model `ResNet152V2` yang telah dilatih pada dataset imagenet. Imagenet adalah sebuah database raksasa yang berisi lebih dari 14 juta gambar.

*   Include_top : parameter ini bernilai boolean. Maksud dari parameter ini apabila kita ingin tetap memakai layer terakhir/layer prediksi dari model resnet. Kita isi 'false' karena kita memakai model resnet untuk memprediksi dataset chessman bukan imagenet.

*   Input_tensor : sesuai namanya parameter ini menspesifikasikan ukuran dari input.





In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2

model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])

model.layers[0].trainable = False

234561536/234545216 [==============================] - 3s 0us/step


Lanjutkan dengan menentukan optimizer, loss, serta metrik yang ingin digunakan pada model.

In [5]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

Terakhir kita dapat melakukan pelatihan model. 

In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              epochs=50,
                              verbose=2)

Epoch 1/50
63/63 - 122s - loss: 8.0820 - accuracy: 0.4629 - val_loss: 1.6914 - val_accuracy: 0.6538
Epoch 2/50
63/63 - 113s - loss: 2.1359 - accuracy: 0.6553 - val_loss: 0.8601 - val_accuracy: 0.7885
Epoch 3/50
63/63 - 112s - loss: 1.2766 - accuracy: 0.7575 - val_loss: 1.9525 - val_accuracy: 0.7115
Epoch 4/50
63/63 - 112s - loss: 0.9128 - accuracy: 0.8016 - val_loss: 1.1135 - val_accuracy: 0.7115
Epoch 5/50
63/63 - 112s - loss: 0.7249 - accuracy: 0.8176 - val_loss: 1.5194 - val_accuracy: 0.5962
Epoch 6/50
63/63 - 110s - loss: 0.5860 - accuracy: 0.8236 - val_loss: 0.7983 - val_accuracy: 0.7885
Epoch 7/50
63/63 - 111s - loss: 0.3871 - accuracy: 0.8737 - val_loss: 0.6960 - val_accuracy: 0.8269
Epoch 8/50
63/63 - 110s - loss: 0.4379 - accuracy: 0.8778 - val_loss: 0.8906 - val_accuracy: 0.7500
Epoch 9/50
63/63 - 112s - loss: 0.2633 - accuracy: 0.9158 - val_loss: 0.9403 - val_accuracy: 0.8462
Epoch 10/50
63/63 - 112s - loss: 0.4335 - accuracy: 0.8998 - val_loss: 1.3164 - val_accuracy: 0.5769

Hasil yang didapat sangat jauh lebih signifikan dibanding melatih sendiri model dari awal. Akurasi dari model meningkat 40% menjadi 70 pada epoch terakhir. Hasil ini sangat luar biasa mengingat setiap kelas pada sampel kita sangat sedikit yaitu kurang dari 100 buah sampel.

# Selamat Anda telah memahami bagaimana menggunakan transfer learning pada Keras. Dengan menggunakan transfer learning prediksi model kita menjadi lebih baik tanpa harus melatih model dari awal sekali untuk mengenali fitur-fitur pada gambar. Jika Anda tertarik mempelajari transfer learning lebih lanjut, kunjungi [tautan berikut](https://www.tensorflow.org/tutorials/images/transfer_learning) yah.